# Describe medallion architecture
Data lakehouses in Fabric are built on the Delta Lake format, which natively supports ACID (Atomicity, Consistency, Isolation, Durability) transactions. Within this framework, the **medallion architecture** is a recommended data design pattern used to organize data in a lakehouse logically. 
- It aims to **improve data quality** as it moves through different layers. 
- The architecture typically has **three layers** – bronze (raw), silver (validated), and gold (enriched), each representing higher data quality levels. 
- Some people also call it a "multi-hop" architecture, meaning that data can move between layers as needed.

This architecture ensures that data is reliable and consistent as it goes through various **checks** and **changes**. It also guarantees that the data is safely stored in a way that makes it **easier** and **faster** to analyze.

The medallion architecture **`complements`** other data organization methods, rather than replacing them.
- You can think of the medallion architecture as the framework for data cleaning, rather than a data architecture or model. 
- It ensures compatibility and flexibility for businesses to adopt its benefits alongside existing data models, allowing you to customize data solutions and preserve expertise while remaining adaptable in the ever-changing data landscape.

<img src="./images/07/lakehouse-medallion.png" alt="Medallion architecture" style="border: 2px solid black; border-radius: 10px;">

## Understand the medallion architecture format
**1. Bronze layer:**
- It's the landing zone for **`all data`**, whether it's structured, semi-structured, or unstructured. 
- The data is stored in its **`original format`**, and no changes are made to it.

**2. Silver layer:**
- It's where you'll **validate** and **refine** your data. 
  - Typical activities in the silver layer include **`combining and merging`** data and **`enforcing data validation rules`** like removing nulls and deduplicating.
  - In the silver layer you're **`cleaning`** your data enough so that everything is in one place and ready to be refined and modeled in the gold layer.
- The silver layer can be thought of as a **central repository** across an organization or team, where data is stored in a consistent format and can be accessed by multiple teams. 

**3. Gold layer:**
- In the gold layer, data undergoes **further refinement** to align with specific business and analytics needs.
  - This could involve **`aggregating`** data to a particular granularity, such as daily or hourly, or **`enriching`** it with external information. 
- Once the data reaches the gold stage, it becomes ready for use by downstream teams, including analytics, data science, or MLOps.

## Customize your medallion architecture
Depending on your organization's specific use case, you may have a need for **`more layers`**. For example, 
- you might have an **`additional "raw" layer`** for landing data in a specific format before it's transformed into the bronze layer.
- you might have a **`"platinum" layer`** for data that's been further refined and enriched for a specific use case. 

Regardless of the names and number of layers, the medallion architecture is flexible and can be tailored to meet your organization's particular requirements.

## Move data across layers in Fabric
There are a few things to consider when deciding how to move and transform data across layers.

- **How much data** are you working with?
- **How complex** are the transformations you need to make?
- **How often** will you need to move data between layers?
- **What tools** are you most comfortable with?
- Understanding the difference between **`data transformation`** and **`data orchestration`** helps you select the right tools for the job within Fabric.
  - **Data transformation:** involves altering the structure or content of data to meet specific requirements.
    - Tools for data transformation in Fabric include **`Dataflows (Gen2)`** and **`notebooks`**.
      - **Dataflows:** are a great option for **`smaller` semantic models** and **`simple` transformations**. 
      - **Notebooks:** are a better option for **`larger` semantic models** and **`more complex` transformations**. Notebooks also allow you to save your transformed data as a managed Delta table in the lakehouse, ready for reporting.
  - **Data orchestration:** refers to the coordination and management of multiple data-related processes, ensuring they work together to achieve a desired outcome
    - The primary tool for data orchestration in Fabric is **`pipelines`**. 
      - A pipeline is a **`series of steps`** that move data from one place to another, in this case, from one layer of the medallion architecture to the next. Pipelines can be automated to run on a schedule or triggered by an event.

# Implement a medallion architecture in Fabric
- **Set up the foundation:** [Create your Fabric lakehouse](https://learn.microsoft.com/en-us/fabric/onelake/create-lakehouse-onelake).
  - You can use **`the same` lakehouse** for **multiple medallion architectures**, or alternatively,
  - You can use **`different` lakehouses** and even **`different` lakehouses `in different` workspaces**, depending on your use case.
- **Design your architecture:** Create your architecture layout, define your layers, and determine how data will flow between them. 
  - The most straightforward implementation is to use Bronze as the raw layer, Silver as the curated layer, and gold as the presentation layer. 
  - Your gold layer should be modeled in a **star schema** and optimized for reporting.

|Question	                  |Bronze	                          |Silver	                  |Gold                                     |
|---------------------------|---------------------------------|-------------------------|-----------------------------------------|
|What happens in that layer?|Ingest raw data                  |Cleanse and validate data|	Additional transformations and modeling |
|What tool is used?         |Pipelines, dataflows, notebooks  |	Dataflows or notebooks  |	SQL analytics endpoint or semantic model|

**1. Ingest data into bronze:** Determine how you'll **`ingest`** data into your bronze layer. You can do this using pipelines, dataflows, or notebooks.

**2. Transform data and load to silver:** Determine how you'll **`transform`** data in your silver layer. You can do this using dataflows or notebooks. Transformations at the silver level should be focused on **`data quality`** and **`consistency`**, not on data modeling.

**3. Generate a gold layer:** Determine how you'll **`generate`** your gold layer(s), what it will contain, and how it will be used.
- The gold layer is where you'll model your data for reporting using a dimensional model. Here you'll **establish relationships**, **define measures**, and incorporate any other elements essential for effective reporting.
- You can have **multiple gold layers** for different audiences or domains. For example, you might have a gold layer for your finance team and a separate gold layer for your sales team. You might also have a gold layer for your data scientists that is optimized for machine learning.
- Depending on your needs, you might also use a Data Warehouse as your gold layer. See Get started with data warehouses in Microsoft Fabric to learn more.

In Fabric, you can transform your data using dataflows or notebooks, and then load it into a gold Delta table in the lakehouse. 
- You can then connect to the Delta table using a **`SQL analytics endpoint`** and use SQL to model your data for reporting. 
- Alternatively, you can use **`Power BI`** to connect to the **`SQL analytics endpoint`** of the gold layer and model your data for reporting.

**Enable downstream consumption:** Determine how you'll enable downstream consumption of your data. You can do this using workspace or item permissions, or by connecting to the **`SQL analytics endpoint`**.

# Query and report on data in your Fabric lakehouse

## Query data in your lakehouse
**1. `SQL analytics endpoint`:** in Fabric enables you to write queries, manage the semantic model, and query data using the new visual query experience.

<img src="./images/07/sql-endpoint-silver.png" alt="SQL analytics endpoint" style="border: 2px solid black; border-radius: 10px;">

<img src="https://files.training.databricks.com/images/icon_note_32.png" alt="Note"> The SQL analytics endpoint operates in **`read-only mode`** over lakehouse delta tables. To modify data in your lakehouse you can use dataflows, notebooks, or pipelines.

**2. `Power BI semantic model`:** you can also create a **Power BI semantic model** in **`Direct Lake mode`** to query data in your lakehouse. 
- When you create a lakehouse, the system also creates an associated default semantic model. The default semantic model is a semantic model with metrics on top of lakehouse data.
- Data analysts connect to the semantic model using Direct Lake mode, in which the semantic model accesses data from a lakehouse. Direct Lake mode **caches often-used data** and refreshes it as required, combining the speed of a semantic model with the up-to-date data from a lakehouse.

![Power BI semantic model](./images/07/dataset-view.png)



## Tailor your medallion layers for different needs
Tailoring medallion layers to **different needs** allows you to optimize data processing and access for specific use cases. 
- By customizing these layers, you can ensure that each layer's structure and organization **`align` with the requirements** of different user groups, **`improving` performance**, **`ease of use`**, and **`data relevance`** for diverse stakeholders.
  - Creating multiple Gold layers tailored for diverse audiences or domains highlights the flexibility of the medallion architecture. Finance, sales, data science – each can have its optimized Gold layer, serving specific analytical requirements.
  - Some applications, third-party tools, or systems require specific data formats. You can utilize your medallion architecture to generate cleansed and properly formatted data.

# Considerations for managing your lakehouse

## Secure your lakehouse
**Secure your lakehouse:** by ensuring that only authorized users can access data. In Fabric, you can do this by **`setting permissions`** at the **workspace** or **item level**.
- **[Workspace permissions](https://learn.microsoft.com/en-us/fabric/get-started/roles-workspaces):** control access to all items within a workspace. 
- **[Item level permissions](https://learn.microsoft.com/en-us/fabric/get-started/share-items):** control access to specific items within a workspace or not in the same workspace.

You can strategically store different layers of your lakehouse in separate workspaces for improved security and efficient capacity management. This approach not only enhances security but also optimizes cost-effectiveness.

- **Security and Access Considerations:** Define who needs access at each layer, ensuring only authorized personnel can interact with sensitive data.
- **Gold Layer Access Control:** Restrict access to the Gold layer for read-only purposes, emphasizing the importance of minimal permissions.
- **Silver Layer Utilization:** Decide whether users will be allowed to build upon the Silver layer, balancing flexibility and security.
- **Bronze Layer Access Control:** Restrict access to the Bronze layer for read-only purposes, emphasizing the importance of minimal permissions.

Sharing of Fabric content should be discussed with your organization's security team to ensure that it aligns with your organization's security policies.

## Considerations for Continuous Integration and Continuous Delivery (CI/CD)
**Designing a Continuous Integration/Continuous Deployment (CI/CD):** process for a lakehouse architecture involves several considerations to ensure a smooth and efficient deployment process.
- Considerations include implementing data quality checks, version control, automated deployments, monitoring, and security measures. 
- Considerations should also include scalability, disaster recovery, collaboration, compliance, and continuous improvement to ensure reliable and efficient data pipeline deployments.
- While some of these are related to processes and practices, others are related to the tools and technologies used to implement CI/CD. Fabric natively provides several tools and technologies to support CI/CD processes.

**`Git integration`** in Microsoft Fabric enables you to integrate development processes, tools, and best practices straight into the Fabric platform.
- Fabric's Git integration enables data teams to back up and version work, revert to previous stages as needed, collaborate with others or work alone using Git branches, and leverage the capabilities of familiar source control tools to manage Fabric items.

<img src="https://files.training.databricks.com/images/icon_note_32.png" alt="Note"> Learn more about Git integration in Fabric in [Introduction to git integration](https://learn.microsoft.com/en-us/fabric/cicd/git-integration/intro-to-git-integration).